In [1]:
require 'nn';
require 'image';
require 'torch';
require 'dataset-mnist'; --- script from https://github.com/torch/demos/blob/master/train-a-digit-classifier/dataset-mnist.lua
mnist = require 'mnist'; -- install mnist loader by "luarocks install https://raw.github.com/andresy/mnist/master/rocks/mnist-scm-1.rockspec"
require 'itorch'

### Load dataset

In [2]:
trainset = mnist.traindataset()
trainset.data = trainset.data:double() -- convert to DoubleTensor
trainset.label:add(1) --class label cannot be 0
testset = mnist.testdataset()
testset.data = testset.data:double() -- convert to DoubleTensor
testset.label:add(1)

print(trainset.data:size())
print(testset.data:size())

itorch.image(trainset.data[10])
print(trainset.label[10])

setmetatable(trainset,  -- to be used in Stochastic Gradient 
    {__index = function(t, i) 
                    return {t.data[i], t.label[i]} 
                end}
);

function trainset:size() -- torch will call trainset:size() during training
    return self.data:size(1) 
end

-- normalization
mean = trainset.data:mean()
trainset.data:add(-mean) 
std = trainset.data:std()
trainset.data:div(std)

 60000
    28
    28
[torch.LongStorage of size 3]

 10000
    28
    28
[torch.LongStorage of size 3]



5	


### Train MNIST using MLP

In [5]:
--model
mlp = nn.Sequential()
mlp:add(nn.View(28*28))
mlp:add(nn.Linear(28*28,128))
mlp:add(nn.ReLU())
mlp:add(nn.Dropout(0.2))
mlp:add(nn.Linear(128,128))
mlp:add(nn.ReLU())
mlp:add(nn.Dropout(0.2))
mlp:add(nn.Linear(128,10))
mlp:add(nn.LogSoftMax())
-- loss function
criterion_mlp = nn.ClassNLLCriterion()

In [ ]:
--training
trainer_mlp = nn.StochasticGradient(mlp,criterion_mlp)
trainer_mlp.learningRate = 0.1
trainer_mlp.maxIteration = 5


trainer_mlp:train(trainset)

# StochasticGradient: training	


In [ ]:
--- training with 9 CPUs and 16G memory
correct = 0
for i=1,10000 do
    groundtruth = testset.label[i]
    prediction = net:forward(testset.data[i])
    confidences, indices = torch.sort(prediction, true)  -- true means sort in descending order
    if groundtruth == indices[1] then
        correct = correct + 1
    end
end

print(correct, 100*correct/10000 .. ' % ')

### Train MNIST using CNN

In [ ]:
cnn = nn.Sequential()
cnn:add(nn.SpatialConvolution(1,32,3,3))
cnn:add(nn.ReLU())
cnn:add(nn.SpatialConvolution(32,32,3,3))
cnn:add(nn.ReLU())
cnn:add(nn.SpatialMaxPooling(2,2,2,2))
cnn:add(nn.SpatialDropout(0.25))
cnn:add(nn.View())
cnn:add(nn.Linear(32*12*12,128))
cnn:add(nn.ReLU())
cnn:add(nn.SpatialDropout(0.5))
cnn:add(nn.Linear(128,10))
cnn:add(nn.LogSoftMax())
criterion = nn.ClassNLLCriterion()

print(cnn)